# 1 Introduction

Our dataset is a set of questions from  the online forum [Stack Exchange](https://stackexchange.com/). Every row has three columns:
 
 1. title of the question
 2. content of the question in HTML format
 3. some tags related to the question
 
We will focus only on the content column. We will try to cluster the documents in a meaningful way so that we can identify similar documents and find groups of related subjects.

The data was found on [Kaggle](https://www.kaggle.com/akshatpathak/text-data-clustering/data). Don't go looking now, because on the webpage it is already splitted into categories. In this exercise, it is the goal to cluster the documents and to find meaningful clusters ourselves, in a unsupervised way.  The data has been merged and mixed.
 

# 2 Exploring and preparing the data

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import time
import numpy as np

In [0]:
questions = pd.read_csv("/content/drive/My Drive/xylosai/clustering/questionData.csv")

In [25]:
len(questions)

87000

In [23]:
questions.head(10)

,title,content,tags
0,Can you pass the customs/passport control if y...,<p>Can one pass the customs or passport contro...,customs-and-immigration alcohol
1,How much alcohol remains in strawberries soake...,<p>I know that the alcohol content of food tha...,fruit food-science alcohol strawberries
2,What can I do to ensure my pool doesn't freeze?,<p>I live about 8 miles north of the Gulf of M...,plumbing pool freezing
3,Attacks on WinRAR volumes reusing same password?,<p>I have quite a few WinRAR volumes that are ...,encryption aes passwords pbkdf-2 key-reuse
4,What can I do about high water pressure that i...,<p>One of our showers has an intermittent leak...,leak water-heater water-pressure
5,Why do humans require vitamin B12 supplementat...,<p>This question came about from reading the c...,biochemistry nutrition
6,Can the attack in https://eprint.iacr.org/2008...,"<p>Can <a href=""https://eprint.iacr.org/2008/4...",implementation collision-resistance sha-1
7,Difference between freezer bag and storage bag,"<p>We accidentally used Ziploc ""storage bags"" ...",storage-method
8,What qualities should I be looking for when ma...,<p>Should it be a thick slice of bread? Should...,eggs bread
9,Why are primes important for encryption,<p>Why are primes so important? Why can't we j...,encryption prime-numbers


In [24]:
questions.drop(columns=["title","tags"],inplace=True)
questions.head()

,content
0,<p>Can one pass the customs or passport contro...
1,<p>I know that the alcohol content of food tha...
2,<p>I live about 8 miles north of the Gulf of M...
3,<p>I have quite a few WinRAR volumes that are ...
4,<p>One of our showers has an intermittent leak...


In [26]:
for question in questions["content"][0:10]:
  print(question)
  print("***"*30)

<p>Can one pass the customs or passport control drunk? </p>

******************************************************************************************
<p>I know that the alcohol content of food that is prepared with alcohol is a tricky study, as evidenced by the fact that food left out overnight stored overnight loses, by <a href="http://www.ochef.com/165.htm" rel="nofollow">one study</a>, 30% of its alcohol content.  Several weeks ago I had some chocolate dipped strawberries that had been soaked in liquor before being dipped in chocolate.  I thought I could taste alcohol, but my dining companion didn't taste it.  So it wasn't a strong flavor.  Is there any information out there on how much alcohol may have been transferred to the strawberries? </p>

******************************************************************************************
<p>I live about 8 miles north of the Gulf of Mexico. It will be in the low teens (below freezing) in the morning. It will be record setting, so I'm

The content is in HTML format with a lot of tags like `` <p></p> or <a></a> ``. We need to extract only the real text from the content body. For this, we use the get_text() method of the BeautifulSoup library.

In [34]:
! pip install beautifulsoup4

In [0]:
from bs4 import BeautifulSoup

Let's test on one example

In [0]:
print(questions["content"][0])
print("***"*30)
print(questions["content"][6])


<p>Can one pass the customs or passport control drunk? </p>

******************************************************************************************
<p>Can <a href="https://eprint.iacr.org/2008/469" rel="nofollow">this attack</a> be parallelized?  It is a $2^{57}$ complexity collision attack on SHA-1 based on disturbance vectors.</p>

<p>If it is practical it is easy on a GPU ($\approx1.1$ GPU-days).  If it is completely sequential it is much more tedious but still possible ($\approx556$ days on a 3GHz CPU).</p>

<p>Both numbers are small enough to beg the question "Why has nobody done it already?".</p>



In [49]:
print(BeautifulSoup(questions["content"][0]).get_text())
print("***"*30)
print(BeautifulSoup(questions["content"][6]).get_text()

Can one pass the customs or passport control drunk? 

******************************************************************************************
Can this attack be parallelized?  It is a $2^{57}$ complexity collision attack on SHA-1 based on disturbance vectors.
If it is practical it is easy on a GPU ($\approx1.1$ GPU-days).  If it is completely sequential it is much more tedious but still possible ($\approx556$ days on a 3GHz CPU).
Both numbers are small enough to beg the question "Why has nobody done it already?".



Let's now apply this on all rows. 

In [53]:
start = time.time()
questions["content_clean"] = questions["content"].apply(lambda x: BeautifulSoup(x).get_text())
end = time.time()
print("operation took {0} seconds".format(end-start))

operation took 24.990045309066772 seconds


In [54]:
questions.head()  

,content,content_clean
0,<p>Can one pass the customs or passport contro...,Can one pass the customs or passport control d...
1,<p>I know that the alcohol content of food tha...,I know that the alcohol content of food that i...
2,<p>I live about 8 miles north of the Gulf of M...,I live about 8 miles north of the Gulf of Mexi...
3,<p>I have quite a few WinRAR volumes that are ...,I have quite a few WinRAR volumes that are all...
4,<p>One of our showers has an intermittent leak...,"One of our showers has an intermittent leak, a..."



# 3 Building the TF-IDF vectors

Sklearn has built-in functionality for calculating TF-IDF vectors. [The documentation is found here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

Read the documentation. Look a the different parameters and decide which ones are relevant and which ones you don't really care about. Are we going to accept the defaults or do we need to change some parameter values?


It is clear that Sklearn builds the vocabulary (the list of all available words) automatically, if no vocabulary is given by the user. For this exercise, we will let Sklearn clear the job for us. 

Scroll down in the documentation and watch the available methods that we can call on the *TfidfVectorizer* object. Follow the links to get more details about the usage of a method. 

the *TfidfVectorizer* object is created with the parameters and fitted with the *fit()* method. When calling *fit()*, the object will internally build the IDF vector. This is the vector we need to transform a word count vector to a TF-IDF vector by multiplying elementwise. After fitting, any word count vector can be transformed into its corresponding TF-IDF vector, using the *transform()* method. 

The creation of a vocabulary, getting the word counts and calculating the IDF vector are completely abstracted away and handled by Sklearn in the background.


When calling *transform()*, a list of text objects is transformed into a TF-IDF weighted matrix of , with a row vector for each document.  These vectors and then used for the clustering


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

We use all default values, except for min_df and max_df. With this setting, we will ignore all words that appear in less than 0.1% of the documents or that appear in more than 99% of the documents. 

In [0]:
vectorizer = TfidfVectorizer(min_df=0.001, max_df=0.99) 

In [63]:
vectorizer.fit(questions["content_clean"])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.99, max_features=None, min_df=0.001,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

We can view our vocabulary with *get_feature_names()*. The first few features are just numbers, after that we get words. If we wanted to, we could tweak the model further so that we have no numbers as features.

In [65]:
print(vectorizer.get_feature_names())
print("size of vocabulary: {0}".format(len(vectorizer.get_feature_names())))

['00', '000', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '1024', '10th', '11', '110', '110v', '12', '120', '120v', '125', '128', '12v', '13', '130', '14', '140', '15', '150', '1500', '15a', '16', '160', '17', '18', '180', '19', '192', '1st', '20', '200', '2000', '2004', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2048', '20a', '21', '22', '220', '220v', '23', '24', '240', '240v', '24v', '25', '250', '255', '256', '26', '27', '28', '29', '2d', '2nd', '2x4', '2x4s', '2x6', '30', '300', '31', '32', '33', '34', '35', '350', '36', '37', '38', '39', '3d', '3rd', '40', '400', '4096', '41', '42', '43', '44', '45', '46', '47', '48', '4th', '4x4', '50', '500', '51', '512', '52', '53', '54', '55', '56', '59', '5mm', '5th', '60', '600', '63', '64', '65', '70', '72', '75', '80', '800', '85', '90', '94', '95', '96', '98', '99', '_p', 'aa', 'ab', 'ability', 'able', 'about', 'above', 'abroad', 'absence', 'absolute', 'absolute

In [0]:
feature_matrix = vectorizer.transform(questions["content_clean"]) 

The result is a Scipy sparse matrix object. It behaves similar like a Numpy array, but it is optimized specifically for sparse matrices. 

Does the shape of the matrix make sense?

In [71]:
print(type(feature_matrix))
print(feature_matrix.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(87000, 5468)


# 4 Clustering

Sklearn supports various clustering algorithms [(see documentation here)](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster)

## 4.1 K-means

First, we will try K-means. For this algorithm, we must decide in advance the number of clusters we want to find. We use the MiniBatchKMeans function. This is basically the same as KMeans, except that it updates the center positions incrementally using small batches of the data, instead of doing a clustering of all data examples in one large batch. 

For a large number of samples, this is much faster. Check yourself by replacing MiniBatchKMeans by KMeans. When using KMeans, the fitting takes very long.

In [0]:
from sklearn.cluster import MiniBatchKMeans, KMeans

In [0]:
kmeans = MiniBatchKMeans(n_clusters = 4,random_state=0)

This could take a while...

In [0]:
cluster_labels = kmeans.fit_predict(feature_matrix)

In [80]:
cluster_labels.shape

(87000,)

In [87]:
print(cluster_labels)
print("unique labels: {0}".format(np.unique(cluster_labels)))

[1 1 3 ... 3 1 2]
unique labels: [0 1 2 3]


We now have a list of cluster labels that map every original question to a cluster label. Since we have not named the clusters, they get a label of 0 to 4.

We add the cluster labels as a column to our original questions dataframe. Then we filter the dataframe in order to display the four clusters.

In [0]:
label_series = pd.Series(cluster_labels)
questions["label"] = label_series

In [90]:
questions["label"].unique()

array([1, 3, 2, 0])

In [89]:
questions.head(10)

,content,content_clean,label
0,<p>Can one pass the customs or passport contro...,Can one pass the customs or passport control d...,1
1,<p>I know that the alcohol content of food tha...,I know that the alcohol content of food that i...,1
2,<p>I live about 8 miles north of the Gulf of M...,I live about 8 miles north of the Gulf of Mexi...,3
3,<p>I have quite a few WinRAR volumes that are ...,I have quite a few WinRAR volumes that are all...,3
4,<p>One of our showers has an intermittent leak...,"One of our showers has an intermittent leak, a...",3
5,<p>This question came about from reading the c...,This question came about from reading the comm...,1
6,"<p>Can <a href=""https://eprint.iacr.org/2008/4...",Can this attack be parallelized? It is a $2^{...,1
7,"<p>We accidentally used Ziploc ""storage bags"" ...","We accidentally used Ziploc ""storage bags"" ins...",1
8,<p>Should it be a thick slice of bread? Should...,Should it be a thick slice of bread? Should it...,1
9,<p>Why are primes so important? Why can't we j...,Why are primes so important? Why can't we just...,1


In [92]:
for label in questions["label"].unique():
  print("First 10 samples of cluster {0}".format(label))
  samples = questions[questions["label"] == label]["content_clean"][0:10]
  print(samples)
  
  

First 10 samples of cluster 1
0     Can one pass the customs or passport control d...
1     I know that the alcohol content of food that i...
5     This question came about from reading the comm...
6     Can this attack be parallelized?  It is a $2^{...
7     We accidentally used Ziploc "storage bags" ins...
8     Should it be a thick slice of bread? Should it...
9     Why are primes so important? Why can't we just...
10    I'm at a loss for what these things are for - ...
18    I took the bathroom mirror off the exterior wa...
20    We have a newly built home but I'm not at all ...
Name: content_clean, dtype: object
First 10 samples of cluster 3
2     I live about 8 miles north of the Gulf of Mexi...
3     I have quite a few WinRAR volumes that are all...
4     One of our showers has an intermittent leak, a...
11    I cannot, for the life of me (no matter what r...
13    Until about 3 or 4 years ago I have been sever...
14    Introduction...\n\nThe dueling scar, also call...
16    Som

Do these clusterings seem accurate? Let's try with a different amount of clusters

In [0]:
kmeans = MiniBatchKMeans(n_clusters = 6,random_state=0)
cluster_labels = kmeans.fit_predict(feature_matrix)
label_series = pd.Series(cluster_labels)
questions["label_6clusters"] = label_series

In [104]:
for label in questions["label_6clusters"].unique():
  print("First 10 samples of cluster {0}".format(label))
  samples = questions[questions["label_6clusters"] == label]["content_clean"][0:10]
  print(samples)

First 10 samples of cluster 4
0     Can one pass the customs or passport control d...
1     I know that the alcohol content of food that i...
5     This question came about from reading the comm...
6     Can this attack be parallelized?  It is a $2^{...
7     We accidentally used Ziploc "storage bags" ins...
9     Why are primes so important? Why can't we just...
10    I'm at a loss for what these things are for - ...
18    I took the bathroom mirror off the exterior wa...
20    We have a newly built home but I'm not at all ...
22    I am dangerously curious to explore places wit...
Name: content_clean, dtype: object
First 10 samples of cluster 5
2     I live about 8 miles north of the Gulf of Mexi...
3     I have quite a few WinRAR volumes that are all...
4     One of our showers has an intermittent leak, a...
8     Should it be a thick slice of bread? Should it...
11    I cannot, for the life of me (no matter what r...
13    Until about 3 or 4 years ago I have been sever...
14    Int

The original data came from 6 distinct classes: biology, cooking, crypto, DIY, robotics and travel. So it should be no surprise that clustering with 6 classes yields better results than 4 classes.

It looks like...
* cluster 0 is about DIY. 
* Cluster 1 is about cooking. 
* Cluster 2 is about travelling
* Cluster 3 has only one sample
* Cluster 4 and 5 are not clear, they appear to be a mix of many





## 4.2 Mean-shift

Instead of first creating a clustering class (MeanShift class), and then calling a *fit()* function on it, we will directly use a function called *mean_shift()*. 

This would also have been possible for K-means, using the *k_means()* function. The options are clear when [reading the docs, where an overview of both classes and functions is given ](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster).

By the way...

[read the docs for mean_shift()!](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.mean_shift.html#sklearn.cluster.mean_shift)

In [0]:
from sklearn.cluster import mean_shift

Running the cell below takes forever. It is clear that mean-shift is computationally more expensive.

In [102]:
cluster_centers, labels = mean_shift(feature_matrix.toarray(),n_jobs=-1)

KeyboardInterrupt: ignored